In [532]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [533]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"STRL"}, 
             {'symbol':"DXPE"}, 
             {'symbol':"TSM"}, 
             {'symbol':"WLDN"}, 
             {'symbol':"SSRM"}, 
             {'symbol':"LRN", 'basis':133.627}, 
             {'symbol':"UNFI", 'basis':20.472}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':100.482}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':57.180}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':0.010}, 
             {'symbol':"PYPL", 'basis':68.207}, 
             {'symbol':"ALL", 'basis':193.027}, 
             {'symbol':"LC", 'basis':7.188}, 
             {'symbol':"QTWO", 'basis':83.760}, 
             {'symbol':"CLS", 'basis':0.010}, 
             {'symbol':"CCL", 'basis':7.890}, 
             {'symbol':"AGX", 'basis':0.010}, 
             {'symbol':"POWL", 'basis':164.655}, 
             {'symbol':"PPC", 'basis':43.297}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE", 'basis':115.105}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM", 'basis':147.990}, 
             {'symbol':"SKYW", 'basis':65.040}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA", 'basis':91.970}, 
             {'symbol':"TWLO", 'basis':87.178}, 
             {'symbol':"PEP"}, 
             {'symbol':"APP", 'basis':221.323}, 
             {'symbol':"TMUS", 'basis':218.823}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':40.612}, 
             {'symbol':"CAAP", 'basis':21.570}
             ]

# Brokerage
brokerage_funds = [{'symbol':"FRSH", 'basis':14.000}, 
                   {'symbol':"PGY", 'basis':28.919}, 
                   {'symbol':"COMM", 'basis':8.320}, 
                   {'symbol':'FINV', 'basis':8.950}, 
                   {'symbol':"LX", 'basis':6.612}, 
                   {'symbol':"WLDN"}, 
                   {'symbol':'BCS', 'basis':18.569}, 
                   {'symbol':'PUK'}, 
                   {'symbol':'PSIX'}, 
                   {'symbol':'NGD', 'basis':4.400}, 
                   {'symbol':'GFI'}, 
                   {'symbol':"CRDO", 'basis':0.010}, 
                   {'symbol':'BKTI', 'basis':40.380}, 
                   {'symbol':"SSRM", 'basis':12.210}, 
                   {'symbol':"UNFI", 'basis':22.716}, 
                   {'symbol':"MFC"}, 
                   {'symbol':"EZPW", 'basis':15.773}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"WFC", 'basis':66.893}, 
                   {'symbol':"ITRN"}, 
                   {'symbol':"PYPL", 'basis':68.332}, 
                   {'symbol':"LC"}, 
                   {'symbol':"QTWO"}, 
                   {'symbol':"CCL", 'basis':0.010}, 
                   {'symbol':"PPC", 'basis':44.580}, 
                   {'symbol':"SYF"}, 
                   {'symbol':"ATGE", 'basis':86.085}, 
                   {'symbol':"SKYW", 'basis':74.583}, 
                   {'symbol':"BLBD"}, 
                   {'symbol':"GM"}, 
                   {'symbol':"OKTA", 'basis':99.535}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"GRBK"}, 
                   {'symbol':"UBER", 'basis':86.860}, 
                   {'symbol':"CAAP"},
                   {'symbol':"NVDA", 'basis':58.908},
                   {'symbol':"QUBT", 'basis':15.469},
                   {'symbol':"RGTI"}, 
                   {'symbol':"FBTC", 'basis':65.965, 'is_etf': True}
                   ]

In [534]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [535]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=.99, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [536]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,FRSH,14.000,12.68,14.15,14.15,14.78,inf,3.752,0.0000,Hold
1,PGY,28.919,29.12,20.42,20.42,22.19,inf,6.570,0.0000,Sell
2,COMM,8.320,7.79,6.45,6.45,6.92,inf,30.360,0.0000,Sell
3,FINV,8.950,8.24,9.03,20.12,9.54,5.93,0.970,0.0336,Hold
4,LX,6.612,6.29,7.05,18.43,7.53,5.88,0.679,0.0289,Hold
6,BCS,18.569,19.10,17.83,45.89,18.27,8.88,0.669,0.0043,Sell
9,NGD,4.400,4.26,4.36,3.09,4.56,21.30,3.062,0.0000,Hold
11,CRDO,0.010,107.56,76.97,6.29,82.77,370.90,27.080,0.0000,Sell
12,BKTI,40.380,37.62,44.53,28.09,48.84,14.41,4.269,0.0000,Hold
13,SSRM,12.210,12.07,11.78,14.83,12.42,29.44,0.772,0.0000,Hold


In [537]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
5,WLDN,0.0,84.00,63.81,31.33,67.38,49.71,4.959,0.0000,Sell
7,PUK,0.0,25.02,23.68,28.05,24.21,14.89,1.833,0.0091,Sell
8,PSIX,0.0,86.99,60.30,21.07,67.28,24.64,23.730,0.0000,Sell
10,GFI,0.0,25.59,22.85,16.59,23.78,18.54,4.403,0.0213,Sell
15,MFC,0.0,30.45,30.78,35.38,31.65,15.94,1.594,0.0530,Hold
19,ITRN,0.0,38.71,37.01,30.24,38.54,13.97,4.024,0.0519,Sell
21,LC,0.0,15.59,11.95,16.40,12.70,24.36,1.272,0.0000,Sell
22,QTWO,0.0,76.53,86.53,4.39,90.51,1275.50,8.180,0.0000,Hold
25,SYF,0.0,67.53,62.15,109.22,64.26,8.21,1.597,0.0151,Sell
28,BLBD,0.0,43.33,40.62,25.81,42.34,13.46,7.180,0.0000,Sell


In [538]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [539]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
5,LRN,133.627,129.26,135.22,84.48,147.82,20.23,3.969,0.0000,Hold
6,UNFI,20.472,27.01,23.45,23.45,25.95,inf,1.007,0.0000,Sell
8,EAT,100.482,155.91,151.78,37.91,165.88,21.68,26.760,0.0000,Hold
9,EZPW,15.460,13.97,13.01,26.82,14.12,10.83,0.859,0.0000,Hold
10,ARQT,15.932,14.29,13.05,13.05,14.50,inf,11.940,0.0000,Hold
11,WFC,57.180,77.78,73.08,100.98,78.51,13.36,1.523,0.0198,Hold
12,ITRN,35.710,38.71,35.51,30.24,38.54,13.97,4.024,0.0519,Sell
13,CRDO,0.010,107.56,73.86,6.29,82.77,370.90,27.080,0.0000,Sell
14,PYPL,68.207,67.11,67.81,58.19,73.29,14.37,3.174,0.0000,Hold
15,ALL,193.027,199.77,186.93,246.77,201.11,9.40,2.391,0.0189,Hold


In [540]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,STRL,0.0,263.05,198.93,88.13,218.40,30.77,9.930,0.0000,Sell
1,DXPE,0.0,110.02,85.81,68.16,94.48,22.97,3.890,0.0000,Sell
2,TSM,0.0,235.21,200.07,92.39,215.10,28.17,7.890,0.0140,Sell
3,WLDN,0.0,84.00,61.23,31.33,67.38,49.71,4.959,0.0000,Sell
4,SSRM,0.0,12.07,11.30,14.83,12.42,29.44,0.772,0.0000,Hold
7,MFC,0.0,30.45,29.54,35.38,31.65,15.94,1.594,0.0530,Hold
23,SYF,0.0,67.53,59.64,109.22,64.26,8.21,1.597,0.0151,Sell
25,BRK-B,0.0,472.84,465.64,624.52,496.92,12.61,1.559,0.0000,Hold
28,BLBD,0.0,43.33,38.98,25.81,42.34,13.46,7.180,0.0000,Sell
29,GM,0.0,52.53,46.79,125.10,50.63,8.02,0.754,0.0096,Sell


In [541]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'}, # Mexico - Communications - mobile network
              {'symbol':'ERIC'}, # Sweden - Communications
              {'symbol':'RDWR'}, # Sweden - Communications
              {'symbol':'PGY'}, # Sweden - Communications
              {'symbol':'STX'}, # Ireland - Technology
              {'symbol':'NVTS'} # Ireland - Technology
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=21)
internationals = build_analysis_table(watch_list, api_key, margin_of_safety=0.95, vwap_days=63)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

internationals_df = pd.DataFrame(internationals, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [542]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.13,23.75,28.08,24.13,9.93,2.344,0.0583,Hold
1,KARO,0,44.92,48.08,19.27,50.16,25.82,7.220,0.4810,Hold
2,QFIN,0,33.65,38.39,72.97,40.08,5.18,1.408,0.0386,Hold
3,TSM,0,235.21,235.15,92.39,240.01,28.17,7.890,0.0140,Hold
4,CCU,0,11.83,12.27,16.35,12.60,12.86,1.396,0.0383,Hold
5,ABEV,0,2.18,2.31,2.48,2.37,12.82,2.059,0.0701,Hold
6,TIMB,0,18.80,18.66,20.61,19.02,14.14,2.018,0.0829,Hold
7,ASR,0,304.03,304.12,280.67,312.83,12.69,3.171,0.0846,Hold
8,RYAAY,0,62.12,59.13,48.77,60.56,14.28,3.897,0.0172,Sell
9,AMX,0,17.96,17.57,15.53,17.94,19.96,2.298,0.0288,Sell


In [543]:
internationals_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.13,22.94,28.08,24.64,9.93,2.344,0.0583,Hold
1,KARO,0,44.92,45.90,19.27,51.01,25.82,7.220,0.4810,Hold
2,QFIN,0,33.65,38.53,72.97,42.22,5.18,1.408,0.0386,Hold
3,TSM,0,235.21,200.07,92.39,215.10,28.17,7.890,0.0140,Sell
4,CCU,0,11.83,12.55,16.35,13.57,12.86,1.396,0.0383,Hold
5,ABEV,0,2.18,2.27,2.48,2.44,12.82,2.059,0.0701,Hold
6,TIMB,0,18.80,17.34,20.61,18.65,14.14,2.018,0.0829,Sell
7,ASR,0,304.03,297.20,280.67,320.48,12.69,3.171,0.0846,Hold
8,RYAAY,0,62.12,52.97,48.77,56.96,14.28,3.897,0.0172,Sell
9,AMX,0,17.96,16.33,15.53,17.55,19.96,2.298,0.0288,Sell


In [544]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, margin_of_safety=0.95, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [545]:
mag7_test_table

[['MSFT', 0, 524.11, 452.69, 146.99, 483.72, 38.45, 11.34, 0.0062, 'Sell'],
 ['AAPL', 0, 202.38, 192.87, 31.69, 207.29, 30.66, 45.62, 0.0049, 'Hold'],
 ['NVDA', 0, 173.72, 137.57, 19.09, 148.54, 56.22, 50.53, 0.0002, 'Sell'],
 ['AMZN', 0, 214.75, 201.11, 83.85, 216.2, 32.79, 6.86, 0, 'Hold'],
 ['GOOG', 0, 189.95, 164.06, 98.25, 177.67, 20.25, 6.33, 0.0052, 'Sell'],
 ['GOOGL', 0, 189.13, 163.06, 98.28, 176.7, 20.16, 6.3, 0.0053, 'Sell'],
 ['META', 0, 750.01, 643.04, 270.76, 692.1, 27.24, 9.66, 0.0027, 'Sell'],
 ['BRK-B', 0, 472.84, 465.64, 624.52, 496.92, 12.61, 1.559, 0, 'Hold'],
 ['TSLA', 0, 302.63, 297.49, 37.27, 328.32, 179.07, 12.63, 0, 'Hold']]

In [546]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,524.11,452.69,146.99,483.72,38.45,11.340,0.0062,Sell
1,AAPL,0,202.38,192.87,31.69,207.29,30.66,45.620,0.0049,Hold
2,NVDA,0,173.72,137.57,19.09,148.54,56.22,50.530,0.0002,Sell
3,AMZN,0,214.75,201.11,83.85,216.20,32.79,6.860,0.0000,Hold
4,GOOG,0,189.95,164.06,98.25,177.67,20.25,6.330,0.0052,Sell
5,GOOGL,0,189.13,163.06,98.28,176.70,20.16,6.300,0.0053,Sell
6,META,0,750.01,643.04,270.76,692.10,27.24,9.660,0.0027,Sell
7,BRK-B,0,472.84,465.64,624.52,496.92,12.61,1.559,0.0000,Hold
8,TSLA,0,302.63,297.49,37.27,328.32,179.07,12.630,0.0000,Hold


In [547]:
# Machine Learning

In [548]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [549]:
# Feature selection and normalization

In [550]:
# Build model

In [551]:
# Train model

In [552]:
# Evaluate model

In [553]:
# Refine model through hyperparameter tuning